In [1]:
import sys
import os.path
sys.path.append(os.path.join('..', 'util'))

import set_compiler
set_compiler.install()

import pyximport
pyximport.install()

import numpy as np
import matplotlib.pyplot as plt

import filtering
from timer import Timer
import threading as th
import logging

%matplotlib inline

Compiling with /usr/local/bin/clang-omp


In [2]:
input_image = np.load('image.npz')['image'].astype(np.float32)

#plt.gray()

#plt.imshow(input_image)
#plt.title('original image')

#plt.figure()
#plt.imshow(input_image[1200:1800, 3000:3500])
#plt.title('before - zoom')


In [3]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.basicConfig(filename='log.txt',
                    level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    )

In [4]:
input_image.shape

(2404, 9184)

In [5]:
events = np.empty( (5, 2), dtype=th._Event )

for i in xrange(5):
    events[i,:] = [th.Event() for _ in xrange(2)]
    
for i in xrange(5):
    print events[i-1,0].is_set()

False
False
False
False
False


In [6]:
def per_thread_median(tmpA,tmpB,thread_ix,evs,I):
    
    for i in range(I):
        logger.debug('Round {}'.format(i))
        
        if i:
            logger.debug('Waiting for i-1={} to be ready'.format(i-1))
            if ix == 0:
                ready_check = evs[thread_ix,i-1].wait() and evs[thread_ix+1,i-1].wait()
            elif ix == evs.shape[0]-1:
                ready_check = evs[thread_ix,i-1].wait() and evs[thread_ix-1,i-1].wait()
            else:
                ready_check = evs[thread_ix-1,i-1].wait() and evs[thread_ix,i-1].wait() and evs[thread_ix+1,i-1].wait() 
        
        logger.debug('Ready to proceed with thread {}, round i={} acquired.'.format(thread_ix,i))

        filtering.median_3x3(tmpA, tmpB, 0, 1)
        # swap direction of filtering
        tmpA, tmpB = tmpB, tmpA

        evs[thread_ix,i].set()
        logger.debug('Ready event for thread {}, round {}'.format(thread_ix,i))
            
def py_median_3x3(image, iterations=10, num_threads=1):
    ''' repeatedly filter with a 3x3 median '''
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)

    events = np.empty( (num_threads, iterations), dtype=th._Event )

    for i in xrange(num_threads):
        events[i,:] = [th.Event() for _ in xrange(iterations)]   
    
    threads = np.empty(num_threads,dtype=th.Thread)
    
    for start in range(num_threads):
        threads[start] = th.Thread(name='t{}'.format(start), 
                                   target=per_thread_median, 
                                   args=(tmpA[start::num_threads,:],
                                         tmpB[start::num_threads,:],
                                         start,
                                         events,
                                         iterations
                                        )
                                  )
        threads[start].start()
        threads[start].join()
        

    return tmpA


In [7]:
def numpy_median(image, iterations=10):
    ''' filter using numpy '''
    for i in range(iterations):
        padded = np.pad(image, 1, mode='edge')
        stacked = np.dstack((padded[:-2,  :-2], padded[:-2,  1:-1], padded[:-2,  2:],
                             padded[1:-1, :-2], padded[1:-1, 1:-1], padded[1:-1, 2:],
                             padded[2:,   :-2], padded[2:,   1:-1], padded[2:,   2:]))
        image = np.median(stacked, axis=2)

    return image

In [8]:
# verify correctness
from_cython = py_median_3x3(input_image, 2, 5)
from_numpy = numpy_median(input_image, 2)


DEBUG:root:Round 0
DEBUG:root:Ready to proceed with thread 0, round i=0 acquired.
DEBUG:root:Ready event for thread 0, round 0
DEBUG:root:Round 1
DEBUG:root:Waiting for i-1=0 to be ready
Exception in thread t0:
Traceback (most recent call last):
  File "/Users/andrew/anaconda/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/Users/andrew/anaconda/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-6-8d77e2428877>", line 8, in per_thread_median
    if ix == 0:
NameError: global name 'ix' is not defined

DEBUG:root:Round 0
DEBUG:root:Ready to proceed with thread 1, round i=0 acquired.
DEBUG:root:Ready event for thread 1, round 0
DEBUG:root:Round 1
DEBUG:root:Waiting for i-1=0 to be ready
Exception in thread t1:
Traceback (most recent call last):
  File "/Users/andrew/anaconda/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/Users/andrew/anaconda/lib/pytho

In [8]:
from_cython

array([[ 157.25259399,  156.25270081,  156.25270081, ...,  152.10049438,
         151.1006012 ,  150.10069275],
       [ 158.25250244,  157.25259399,  157.25259399, ...,  152.10049438,
         152.10049438,  151.1006012 ],
       [ 157.25259399,  157.25259399,  157.25259399, ...,  152.23150635,
         152.10049438,  150.10069275],
       ..., 
       [ 150.19599915,  150.19599915,  153.65789795, ...,   20.16139984,
          22.16119957,   23.87290001],
       [ 148.65840149,  148.65840149,  151.65809631, ...,   22.04719925,
          23.16110039,   24.04700089],
       [ 148.65840149,  148.89720154,  150.65820312, ...,   22.04719925,
          23.04710007,   23.04710007]], dtype=float32)

In [9]:
from_numpy

array([[ 159.25239563,  159.25239563,  156.89360046, ...,  152.10049438,
         150.10069275,  150.10069275],
       [ 159.25239563,  156.89360046,  156.89360046, ...,  152.10049438,
         150.10069275,  149.10079956],
       [ 156.25270081,  156.25270081,  156.89360046, ...,  150.10069275,
         148.10090637,  148.10090637],
       ..., 
       [ 146.65859985,  148.65840149,  151.2559967 , ...,   20.04739952,
          22.04719925,   22.16119957],
       [ 148.65840149,  148.65840149,  151.2559967 , ...,   21.04730034,
          22.04719925,   23.04710007],
       [ 148.65840149,  149.65829468,  150.65820312, ...,   21.04730034,
          22.04719925,   23.04710007]], dtype=float32)

In [ ]:

assert np.all(from_cython == from_numpy)

with Timer() as t:
    new_image = py_median_3x3(input_image, 10, 8)

pylab.figure()
pylab.imshow(new_image[1200:1800, 3000:3500])
pylab.title('after - zoom')

print("{} seconds for 10 filter passes.".format(t.interval))
pylab.show()

In [24]:

def consumer(cond1,cond2,cond3,id):

    # wait for the condition and use the resource

    logging.debug('Starting consumer thread')

    with cond1, cond2, cond3:
        cond1.wait()
        cond2.wait()
        cond3.wait()
    logging.debug('Lock 1 is available to consumer, {} speaking here'.format(id))
    '''
    with cond2:
        cond2.wait()
    logging.debug( 'Lock 2 is available to consumer, {} speaking here'.format(id) )
    
    with cond3:
        cond3.wait()
    
    logging.debug('Lock 3 is available to consumer, {} speaking here'.format(id))
    '''
def producer(cond, id):

    # set up the resource to be used by the consumer

    logging.debug('Starting producer thread {}'.format(id))
    logging.debug('Making resource available')
    
    with cond:
        cond.notify()


c1 = th.Condition()
c2 = th.Condition()
c3 = th.Condition()

consumer1 = th.Thread(name='c1', target=consumer, args=(c1,c2,c3,'c1'))
#consumer2 = threading.Thread(name='c2', target=consumer, args=(c1,c2,c3,'c2'))

p1 = th.Thread(name='p1', target=producer, args=(c1,'p1'))
p2 = th.Thread(name='p2', target=producer, args=(c2,'p2'))
p3 = th.Thread(name='p3', target=producer, args=(c3,'p3'))

consumer1.start()
#consumer2.start()

p1.start()
p2.start()
p3.start()

DEBUG:root:Starting consumer thread
DEBUG:root:Starting producer thread p1
DEBUG:root:Making resource available
DEBUG:root:Starting producer thread p2
DEBUG:root:Making resource available
DEBUG:root:Starting producer thread p3
DEBUG:root:Making resource available
DEBUG:root:Lock 1 is available to consumer, c1 speaking here
